In [1]:
from dotenv import load_dotenv
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, EMA,
    EmpyrealOrderlySDK,
)
from emp_orderly_types import PerpetualAssetType, Interval

load_dotenv()

/home/ademola/.cache/pypoetry/virtualenvs/balo-orderly-submission-oa8W0BMU-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [3]:
print(wallet.address)

0x4Bc244f1c09C5C5c15DC5959aCFc0632fe87f100


In [4]:
print(await wallet.balance())

0


In [5]:
class EmaCross(Strategy):
    """Exponential Moving Average Death/Golden Cross Strategy.
    
    order_size: The amount of order that can be 
        made at a time.
    days_length_1: 12 days length for 
        calculating 12 days EMA.
    days_length_2: 26 days length for 
        calculating 26 days EMA.
    length: The number of days the Strategy is 
        intended to run for.
    """

    order_size: float = 0.999
    days_length_1: int = 12
    days_length_2: int = 26
    
    @classmethod
    def update_lags(cls, days_length_1, days_length_2):
        cls.days_length_1 = days_length_1
        cls.days_length_2 = days_length_2

    def init(self):
        close = self.data.close
        self.ema_days_1 = self.I(EMA, close, self.days_length_1)
        self.ema_days_2 = self.I(EMA, close, self.days_length_2)

    def next(self):
        if (
            crossover(self.ema_days_1, self.ema_days_2)
        ):
            self.position.close()
            self.buy(size=self.order_size)
        elif (
            crossover(self.ema_days_2, self.ema_days_1)
        ):
            self.position.close()
            self.sell(size=self.order_size)


tester = EmpOrderly(
    cash=1000000,
    commission=.0000001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(EmaCross)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.DOGE,
)

# backtest
tester.backtest()

Start                     2024-09-25 03:15:00
End                       2024-10-02 02:20:00
Duration                      6 days 23:05:00
Exposure Time [%]                   83.000997
Equity Final [$]               1386286.460388
Equity Peak [$]                1441084.181271
Return [%]                          38.628646
Buy & Hold Return [%]                3.165949
Return (Ann.) [%]            296511907.177803
Volatility (Ann.) [%]        648492406.196775
Sharpe Ratio                         0.457233
Sortino Ratio                 25408427.207067
Calmar Ratio                  57027860.414931
Max. Drawdown [%]                   -5.199422
Avg. Drawdown [%]                   -1.329384
Max. Drawdown Duration        1 days 18:55:00
Avg. Drawdown Duration        0 days 04:33:00
# Trades                                   49
Win Rate [%]                        34.693878
Best Trade [%]                      17.232464
Worst Trade [%]                      -1.07677
Avg. Trade [%]                    

In [6]:
# plot
tester.plot(show_price_data=False)

GridPlot(id='p1316', ...)

In [7]:
plt.show()

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ademola/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.
